# Project: Sensitivity of coupon-bearing Treasury notes and bonds
Malkiel proposed five theorems that govern the price of fixed-income debt securities, e.g., Treasury Bills, Notes, and Bonds, as a function of the duration and yield values:

* [Malkiel, B. G. (1962). Expectations, Bond Prices, and the Term Structure of Interest Rates. The Quarterly Journal of Economics, 76(2), 197–218. https://doi.org/10.2307/1880816](https://www.jstor.org/stable/1880816https://www.jstor.org/stable/1880816)

In this project, let's develop a simulation to validate `Theorem 5`:
> `Theorem 5`: The higher the coupon carried by the bond, the smaller the percentage price fluctuation for a given change in yield

## Learning objectives
The objective of this project is to familiarize students with computing the prices of Treasury Bills, Notes, and Bonds and to compare simulations of the change in the prices of these instruments with the five theorems proposed by Malkiel:

* __Objective 1__: Compute the price, visualize the cashflows for a coupon-bearing Treasury note
    * `TODO`: Build an instance of `MyUSTreasuryCouponSecurityModel`, and compute the price and other data for the note
        * `Check`: Are the computed and observed note prices `similar`?
    * `TODO`: Visualize the `nominal,` `discounted,` and `cumulative` cash flow
* __Objective 2__: Simulate Theorem 5 of Malkiel for a coupon-bearing Treasury note
    * `Discussion question`: Do your simulations support `Theorem 5?`

## Setup
We load the [VLQuantitativeFinancePackage.jl](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl) package and several other external [Julia](https://julialang.org/downloads/https://julialang.org/downloads/) packages, as well as some helper code and custom Types (see below), by calling the [include(...)](https://docs.julialang.org/en/v1/manual/code-loading/https://docs.julialang.org/en/v1/manual/code-loading/) command on the file `Include.jl`:

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Installed Compat ─ v4.11.0
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Project.toml`
    Updating `~/Desktop/julia_work/CHEME-130-eCo

### Types
In addition to external packages, `Include.jl` also loads some [problem-specific custom types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of Treasury instrument pricing. 
* `MyUSTreasuryCouponSecurityModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$, the effective market rate of interest rate $\bar{r}$, the coupoun rate $c$ and the number of coupon payments per year $\lambda$ for treasury note and bonds that pay a coupoun. You construct a `MyUSTreasuryCouponSecurityModel` instance using the `build` method described below.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let your code know which compounding model to use.

## Objective 1: Compute the price, visualize the cashflows for a coupon-bearing Treasury note
Unlike zero-coupon Treasury bills, which have only two cash flow events (you give money to the Treasury, and you receive the face (par) value of the `T-note` at maturity), coupon-bearing Treasury securities are more complicated because of the periodic coupon payments. Thus, it's helpful to visualize the cash flow events of notes and bonds. 
* We begin by building an instance of the `DiscreteCompoundingModel()` type and store this discount model in the `discount_model` variable:

In [2]:
discount_model = DiscreteCompoundingModel();

### TODO: Build an instance of `MyUSTreasuryCouponSecurityModel`, and compute the price
Next, let's build an instance of the `MyUSTreasuryCouponSecurityModel` type using the `build(...)` method. In this example, we'll compute the price and cashflow for a `T = 7-yr` bond, with a coupon rate of `c = 1.375%`, a yield (discount rate) `rate = 1.461%`, two coupon payments per year, i.e., $\lambda = 2$ and a face (par) value of $V_{P}$ = `100 USD`. Let's store the result in the `test_note` variable. 
* The price value reported on [TreasuryDirect.gov](https://www.treasurydirect.gov/marketable-securities/understanding-pricing/#id-for-more-detailed-formulas-and-useful-tables-264977) for this note is $V_{B}$ = `99.4299 USD`.
* Similar to zero-coupon `T-bills,` we'll use a `short-cut` syntax that relies on the [Julia piping |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) and some syntax sugar to compute the coupon-bearing price, and discount factors and the cashflow for the instrument. 

In [6]:
T = 7
r̄ = 0.01461
c = 0.01375

test_note = build(MyUSTreasuryCouponSecurityModel, (
    T = T, rate = r̄, coupon = c, λ = 2, par = 100.0
)) |> discount_model;

We have populated the `MyUSTreasuryCouponSecurityModel` instance stored in the `test_note` variable
* Let's pull the `price,` `discount,` and `cashflow` fields from `test_note` and store them in the `nominal_computed_price,` `cashflow,` and `discount` variables

In [23]:
nominal_computed_price = test_note.price;
cashflow = test_note.cashflow;
discount = test_note.discount;
println("This computed note price = $(nominal_computed_price) USD")

This computed note price = 99.42973596186266 USD


#### Check: Are the computed and observed prices `similar`?
Use the [isapprox function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) combined with the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check the `similarity` of the computed, and observed note price. If the price values are different beyond `rtol = 1e-4`, a `false` result is generated, and an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is thrown.

In [10]:
observed_bond_price = 99.4299;
@assert isapprox(observed_bond_price, nominal_computed_price; rtol = 1e-4)

### TODO: Visualize the `nominal,` `discounted,` and `cumulative` cash flow
Finally, let's build a table holding the `nominal,` `discounted,` and `cumulative` cash flow for this bond using the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl). We'll iterate through each period using a `for` loop and populate the `bond_data_table` variable. For each iteration of the loop, we will:
* We access values for the discount and cashflow in period `i` and compute the cumulative cashflow in the `sumvalue` variable
* We then store these data along with the nominal cash flow for each period in the `bond_data_table` array and display the data in the `bond_data_table` variable by calling the `pretty_table(...)` function (with optional values for the `header,` and `tf` arguments)

In [12]:
number_of_periods = length(cashflow)
bond_data_table = Array{Any,2}(undef, number_of_periods, 5);
sumvalue = 0.0;
for i ∈ 0:(number_of_periods - 1)
    
    discount_value = discount[i]
    payment = cashflow[i];
    sumvalue += payment;

    bond_data_table[i+1,1] = i;
    bond_data_table[i+1,2] = discount_value;
    bond_data_table[i+1,3] = discount_value*payment;
    bond_data_table[i+1,4] = payment;
    bond_data_table[i+1,5] = sumvalue;
end
pretty_table(bond_data_table; 
    header=["Period", "Discount factor", "Nominal cashflow", "Discounted cashflow", "Cumulative cashflow"], 
    tf = tf_simple)

========= ================= ================== ===================== ======================
  Period   Discount factor   Nominal cashflow   Discounted cashflow   Cumulative cashflow 
========= ================= ================== ===================== ======================
       0               1.0           -99.4297              -99.4297              -99.4297
       1            1.0073             0.6875              0.682514              -98.7472
       2           1.01466             0.6875              0.677565              -98.0697
       3           1.02208             0.6875              0.672651               -97.397
       4           1.02954             0.6875              0.667773              -96.7292
       5           1.03706             0.6875               0.66293              -96.0663
       6           1.04464             0.6875              0.658123              -95.4082
       7           1.05227             0.6875               0.65335              -94.7548
     

## Objective 2: Simulate Theorem 5 of Malkiel for a coupon-bearing Treasury note
To simulate the impact of changes in the interest and coupon rate, let's perturb the effective nominal interest rate $\bar{r}$, with all other values held constant, i.e., generate a new rate of the form $\bar{r}\leftarrow\beta\cdot\bar{r}$, where $\beta$ is a perturbation value; if $\beta<1$ the perturbed interest rate is _less than_ the nominal rate, if $\beta=1$ the perturbed interest rate is _equals_ the nominal rate, and if $\beta>1$ the perturbed interest rate is _greater than_ the nominal rate.

* First, specify the number of perturbation values in the `number_of_samples` variable; for this simulation, let `number_of_samples = 3`
* Next, specify the lower bound in the `β₁` variable and the upper bound in the `β₂` variable; let `β₁ = 0.8` and `β₂ = 1.2.`
* Finally, we compute the perturbation array (stored in the `β`-variable) using the [range function](https://docs.julialang.org/en/v1/base/math/#Base.range) in combination with the [Julia pipe |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping), and the [collect function](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}), which converts a range type to a collection, i.e., an array

In [13]:
number_of_samples = 3;
β₁ = 0.8;
β₂ = 1.2;
β = range(β₁, stop = β₂, length = number_of_samples) |> collect;

Your job is to complete the implementation of the simulation of `Theorem 5` started below, and analyze the simulation results.
*  `Hint`: check out the [deepcopy function](https://docs.julialang.org/en/v1/base/copy/#Base.deepcopy) in `Julia`

In [20]:
simulation_results_array = Array{Float64,2}(undef, number_of_samples, number_of_samples);
for i ∈ eachindex(β)
    β_outer = β[i]
    
    model = deepcopy(test_note);
    model.coupon = β_outer*test_note.coupon;
    
    for j ∈ eachindex(β)
        
        β_inner = β[j];
        model.rate = β_inner*test_note.rate;
        perturbed_price = model |> discount_model |> x-> x.price
        simulation_results_array[i,j] = ((perturbed_price - nominal_computed_price)/nominal_computed_price)*100
    end
end
simulation_results_array;

Fill me in

In [22]:
# build a pretty table to display the results -
(R,C) = size(simulation_results_array)
pretty_table_data = Array{Any,2}(undef, R, C+1)

# first col holds labels -
for i ∈ 1:R
    if (i == 1)
        pretty_table_data[i,1] = "-20% coupon";
    elseif (i == 3)
        pretty_table_data[i,1] = "+20% coupon";
    else
        pretty_table_data[i,1] = "nominal coupon";
    end
end

for i = 1:R
    for j = 1:C
        pretty_table_data[i,j+1] = simulation_results_array[i,j]
    end
end

header_data = (["", "-20% yield", "nominal yield", "+20% yield"])
pretty_table(pretty_table_data, header=header_data, tf=tf_simple)

================= ============ =============== =============
                   -20% yield   nominal yield   +20% yield 
================= ============ =============== =============
     -20% coupon     0.109757        -1.83398     -3.73638
  nominal coupon      1.96352             0.0     -1.92189
     +20% coupon      3.81729         1.83398    -0.107394
================= ============ =============== =============


### Discussion question: Do your simulations support the `Theorem 5`?
1. From the table above, do the `Theorem 5` simulations support Malkiel's hypothesis, i.e., that high coupon instruments are less sensitive to changes in yield (discount rate)?

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.